In [1]:
import en_core_web_md
import pandas as pd
from tqdm import tqdm, tqdm_pandas
import qgrid

pd.set_option('display.max_colwidth', 300)

In [2]:
tqdm.pandas()

In [3]:
nlp = en_core_web_md.load()

In [4]:
cleaned = pd.read_csv('dataset/phrases_cleaned_1522422847595.csv', usecols=['id', 'phrase'])
cleaned = cleaned.assign(doc=cleaned['phrase'].progress_apply(nlp))

100%|██████████| 287/287 [00:03<00:00, 84.42it/s]


In [5]:
SPLITS = {'who', ',','and'}
cleaned = cleaned.assign(idxs=cleaned['doc'].progress_apply(lambda doc: [tok.idx for tok in doc if tok.lemma_ in SPLITS]))
splitted_again = []
with_indicies = []
for i,row in cleaned.iterrows():
    idxs = set(row['idxs'])
    phrase = row['phrase']
    subphrases = []
    subphrase = ''
    for i,ch in enumerate(phrase):
        if i in idxs:
            subphrases.append(subphrase)
            subphrase = ch
        else:
            subphrase += ch
    splitted_again.append(subphrases)
    with_indicies.append([[tok.idx,tok] for tok in row['doc']])
        

100%|██████████| 287/287 [00:00<00:00, 37923.42it/s]


In [17]:
def add_dollars(phrase):
    doc = nlp(phrase)
    new_phrase = phrase
    acc = 0
    for tok in doc:
        if tok.lemma_ in SPLITS:
            new_phrase = new_phrase[:tok.idx+acc] + '$$' + new_phrase[tok.idx+acc:]
            acc += 2
    return new_phrase

qdf = pd.DataFrame({'text': cleaned['phrase'].apply(add_dollars)})
widget = qgrid.show_grid(qdf)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [19]:
widget.get_changed_df().to_csv('')

,text
0,Find people $$who wrote posts mentioning Stacy Vaughn $$and liked any post 9 times $$and clicked on ad with 'blackout' in it
1,Find all attendees $$who wrote at least one post with mention of Stacy Vaughn $$and liked any post 9 or more times $$and clickd on ad with text 'blackout' in it
2,Find all attendees $$who wrote a post about Stacy Vaughn $$and have at least nine likes $$and clickd on add that contained 'blackout'
3,Find all attendees $$who clicked on ad about blackout $$and also liked post 9 times $$and wrote post with Stacy Vaughn's mention
4,Find client $$who wrote Stacy Vaughn's post $$and liked it 9 times $$as well as clicked on ad that contains 'blackout'
5,Attendees $$who rated page at least 4 times $$and answered 'Yes' in a poll $$and downloaded 'exhibitor' type page file
6,Find attendees $$who rated any page four times $$and participated in poll and answered yes to any question $$as well as downloaded a file from page with 'exhibitor' type
7,Find people $$who rated page 4 time and answered yes in poll $$and downloaded a file from exhibitor page
8,Find attendees $$who rated page 4 or more times $$and in poll answered affirmative $$and got file from 'exhibitor' type page
9,Find attendees $$who rated page four times $$as well as participated in poll $$and answered positively $$and downloaded any file from exhibitor page


In [13]:
splitted = pd.read_csv('dataset/phrases_splitted_1522422847595.csv')
splitted.head()

,id,splitted_phrase
0,e82bae79-8891-4244-9745-bcb07935845c,"[""wrote posts mentioning Stacy Vaughn"", ""liked any post 9 times"", ""clicked on ad with 'blackout' in it""]"
1,e82bae79-8891-4244-9745-bcb07935845c,"[""wrote at least one post with mention of Stacy Vaughn"", ""liked any post 9 or more times"", ""clickd on ad with text 'blackout' in it""]"
2,e82bae79-8891-4244-9745-bcb07935845c,"[""wrote a post about Stacy Vaughn"", ""at least nine likes"", ""clickd on add that contained 'blackout'""]"
3,e82bae79-8891-4244-9745-bcb07935845c,"[""clicked on ad about blackout"", ""also liked post 9 times"", ""wrote post with Stacy Vaughn's mention""]"
4,e82bae79-8891-4244-9745-bcb07935845c,"[""wrote Stacy Vaughn's post"", ""liked it 9 times"", ""clicked on ad that contains 'blackout'""]"
